# Input Files and Imports

In [1]:
import sys
print(sys.path)


['C:\\Users\\Mariana Saavedra\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'C:\\Users\\Mariana Saavedra\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'C:\\Users\\Mariana Saavedra\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'C:\\Users\\Mariana Saavedra\\AppData\\Local\\Programs\\Python\\Python311', 'c:\\Users\\Mariana Saavedra\\Desktop\\Tese\\.venv', '', 'c:\\Users\\Mariana Saavedra\\Desktop\\Tese\\.venv\\Lib\\site-packages', 'c:\\users\\mariana saavedra\\desktop\\tese', 'c:\\Users\\Mariana Saavedra\\Desktop\\Tese\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\Mariana Saavedra\\Desktop\\Tese\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\Mariana Saavedra\\Desktop\\Tese\\.venv\\Lib\\site-packages\\Pythonwin']


In [2]:
#!/usr/bin/env python

# # Imports

# Standard libraries
import os
import sys

# Numerical and data processing libraries
import numpy as np
import pandas as pd
import pickle

# TensorFlow and Keras
from tensorflow import keras

# Custom libraries
from libs.paths import data_folder, results_folder, models_folder
from libs import feature_extraction_lib as ftelib
from libs.feature_extraction_lib_extension import process_pcg_signals_from_pkl
from libs.label_mappings import get_label_meaning

from libs import unet_model as unet

# # Input files

signal = 'pcg'
label_x = 0  # Replace with your chosen label for x
label_y = 2  # Replace with your chosen label for y

label_string = get_label_meaning(signal, label_x, label_y)

print(label_string)  # Output: 'S1S2' for PCG, 'baseline segmento QRS' for ECG

# Load Estimates
data_file_path = results_folder / f"{label_string}_estimates_and_annotations.csv"
df = pd.read_csv(data_file_path)  # Use read_csv instead of read_pickle for CSV files

print(df.head())

S1S2
                                                   0  \
0  [(0.0, 0.04), (18.42, 18.52), (20.06, 20.16), ...   
1  [(0.0, 0.04), (18.42, 18.52), (20.06, 20.16), ...   
2  [(0.88, 1.0), (2.22, 2.2600000000000002), (4.9...   
3  [(0.88, 1.0), (2.22, 2.2600000000000002), (4.9...   
4  [(0.0, 0.06), (5.7, 5.84), (12.86, 12.98), (21...   

                                                   2  \
0  [(17.54, 17.64), (19.7, 19.72), (20.3, 20.32),...   
1  [(17.54, 17.64), (19.7, 19.72), (20.3, 20.32),...   
2  [(0.0, 0.06), (1.2, 1.26), (3.9, 3.9), (19.44,...   
3  [(0.0, 0.06), (1.2, 1.26), (3.9, 3.9), (19.44,...   
4  [(2.8000000000000003, 2.86), (5.96, 6.04), (20...   

                               Intervals_start_times  \
0  [17.54, 1.2799999999999976, 0.240000000000002,...   
1  [17.54, 1.2799999999999976, 0.240000000000002,...   
2  [0.31999999999999995, 1.6799999999999997, 14.4...   
3  [0.31999999999999995, 1.6799999999999997, 14.4...   
4  [2.8000000000000003, 0.259999999999999

In [ ]:
import pickle

# Define the path where the file is stored
predictions_pickle_path = '/content/drive/MyDrive/Tese/Daniel_PCG_UNET/Resultados Mariana/reconstructed_labels.pkl'

# Load the pickle file
with open(predictions_pickle_path, 'rb') as file:
    predictions = pickle.load(file)

# Print or check the loaded data
print(type(predictions))  # Check the type of loaded object
print(predictions[:5])  # Print the first few elements (if it's a list or array)

<class 'list'>
[array([[0.35381019, 0.2540606 , 0.19457965, 0.19754964],
       [0.38906765, 0.17533119, 0.26615313, 0.16944799],
       [0.35643852, 0.18448004, 0.28323871, 0.17584267],
       ...,
       [0.29280463, 0.16435444, 0.19531116, 0.34752983],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]]), array([[0.01414598, 0.05432292, 0.01166636, 0.91986471],
       [0.0070835 , 0.0327984 , 0.00253604, 0.957582  ],
       [0.00808579, 0.04174281, 0.00207181, 0.94809949],
       ...,
       [0.05297251, 0.09965677, 0.59228003, 0.25509074],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]]), array([[0.00636502, 0.01882447, 0.03067411, 0.9441365 ],
       [0.00358492, 0.00824108, 0.01072847, 0.97744554],
       [0.0043205 , 0.00770316, 0.00896073, 0.97901553],
       ...,
       [0.00194044, 0.00789562, 0.00138208, 0.98878181],
       [0.        , 0.        , 

#Automatic S1S2 extraction

In [ ]:
import pandas as pd

df_probabilities = pd.DataFrame({
    'S1': [signal[:, 0] for signal in predictions],
    'S2': [signal[:, 2] for signal in predictions]
})

# Access S1 probabilities for signal 5
df_probabilities.iloc[5]['S1']

array([0.22645219, 0.32340893, 0.36696267, ..., 0.        , 0.        ,
       0.        ])

## Moving Average

In [ ]:
# Define the function for processing S1 and S2 columns
def apply_moving_average(df, window_size):
    """
    Apply the moving average filter to S1 and S2 probabilities in the DataFrame.

    Parameters:
    ----------
    df : pandas.DataFrame
        Input DataFrame containing S1 and S2 columns with probabilities as lists/arrays.
    window_size : int
        The size of the moving average window.

    Returns:
    -------
    pandas.DataFrame
        Updated DataFrame with filtered S1 and S2 probabilities.
    """
    df['S1_filtered'] = df['S1'].apply(lambda x: pplib.moving_average(np.array(x), window_size))
    df['S2_filtered'] = df['S2'].apply(lambda x: pplib.moving_average(np.array(x), window_size))
    return df

# Apply the function to your DataFrame
window_size = 5  # Adjust this as needed (must be a positive odd integer)
df_probabilities = apply_moving_average(df_probabilities, window_size)

# Display the updated DataFrame
print(df_probabilities.head())

                                                  S1  \
0  [0.35381019115448, 0.3890676498413086, 0.35643...   
1  [0.014145980589091778, 0.007083502132445574, 0...   
2  [0.0063650161027908325, 0.0035849222913384438,...   
3  [0.04696093872189522, 0.03495466709136963, 0.0...   
4  [0.006586980074644089, 0.002950427122414112, 0...   

                                                  S2  \
0  [0.19457964599132538, 0.26615312695503235, 0.2...   
1  [0.011666355654597282, 0.0025360428262501955, ...   
2  [0.03067411296069622, 0.010728465393185616, 0....   
3  [0.10874853283166885, 0.10066991299390793, 0.1...   
4  [0.008311466313898563, 0.0020235825795680285, ...   

                                         S1_filtered  \
0  [0.3448824810981751, 0.3245721089839935, 0.289...   
1  [0.012434091232717038, 0.02084208713844419, 0....   
2  [0.005318782636895776, 0.0047213813662529, 0.0...   
3  [0.044060727208852776, 0.042983179837465295, 0...   
4  [0.005383932376280428, 0.005270281061530114

## First Approach: peak of the probabilities

In [ ]:
from scipy.signal import find_peaks

def extract_positions(probabilities, threshold=0.5):
    """
    Extract positions of peaks above the threshold for a given probability array.

    Parameters:
    ----------
    probabilities : numpy.ndarray
        Filtered probability array (e.g., S1 or S2 probabilities).
    threshold : float
        Threshold value to consider a point as a peak.

    Returns:
    -------
    list
        Positions of detected peaks above the threshold.
    """
    # Find peaks in the probability array
    peaks, _ = find_peaks(probabilities, height=threshold)
    return peaks.tolist()

# Apply the function to S1 and S2 filtered probabilities
df_probabilities['S1_positions'] = df_probabilities['S1_filtered'].apply(
    lambda x: extract_positions(x, threshold=0.5)
)

df_probabilities['S2_positions'] = df_probabilities['S2_filtered'].apply(
    lambda x: extract_positions(x, threshold=0.5)
)

# Example of S1-S2 interval calculation
def calculate_s1s2_intervals(row):
    """
    Calculate the intervals between S1 and S2 peaks.

    Parameters:
    ----------
    row : pandas.Series
        A row of the DataFrame containing S1_positions and S2_positions.

    Returns:
    -------
    list
        S1-S2 intervals for the current signal.
    """
    s1_positions = row['S1_positions']
    s2_positions = row['S2_positions']
    intervals = []
    for s1 in s1_positions:
        # Find the closest S2 that occurs after the current S1
        s2_after = [s2 for s2 in s2_positions if s2 > s1]
        if s2_after:
            intervals.append(s2_after[0] - s1)
    return intervals

# Apply interval calculation to the DataFrame
df_probabilities['S1S2_intervals'] = df_probabilities.apply(calculate_s1s2_intervals, axis=1)

# Display the DataFrame with extracted positions and intervals
print(df_probabilities[['S1_positions', 'S2_positions', 'S1S2_intervals']].head())

                                        S1_positions  \
0  [110, 141, 359, 761, 788, 851, 896, 940, 1060,...   
1  [9, 46, 91, 121, 228, 278, 322, 463, 511, 575,...   
2  [20, 66, 110, 156, 201, 247, 308, 337, 380, 42...   
3  [23, 100, 128, 203, 245, 404, 493, 569, 649, 6...   
4  [10, 40, 68, 98, 128, 157, 186, 218, 247, 373,...   

                                        S2_positions  \
0                                             [1431]   
1  [24, 61, 134, 294, 893, 1077, 1124, 1306, 1352...   
2  [37, 82, 173, 218, 263, 352, 397, 442, 532, 57...   
3       [663, 818, 864, 894, 1057, 1186, 1216, 1287]   
4  [22, 51, 80, 110, 139, 168, 198, 230, 260, 385...   

                                      S1S2_intervals  
0  [1321, 1290, 1072, 670, 643, 580, 535, 491, 37...  
1  [15, 15, 43, 13, 66, 16, 571, 430, 382, 318, 1...  
2  [17, 16, 63, 17, 17, 16, 44, 15, 17, 17, 61, 1...  
3  [640, 563, 535, 460, 418, 259, 170, 94, 14, 13...  
4  [12, 11, 12, 12, 11, 11, 12, 12, 13, 12, 12, 1..

In [ ]:
def calculate_s1s2_intervals(row, min_interval, max_interval):
    """
    Calculate the intervals between S1 and S2 peaks, with restrictions on valid ranges.

    Parameters:
    ----------
    row : pandas.Series
        A row of the DataFrame containing S1_positions and S2_positions.
    min_interval : int
        Minimum acceptable interval in samples.
    max_interval : int
        Maximum acceptable interval in samples.

    Returns:
    -------
    list
        Valid S1-S2 intervals for the current signal.
    """
    s1_positions = row['S1_positions']
    s2_positions = row['S2_positions']
    intervals = []

    # Ensure there are both S1 and S2 peaks
    if not s1_positions or not s2_positions:
        return intervals  # Return empty if no valid peaks

    for s1 in s1_positions:
        # Find the closest S2 that occurs after the current S1
        s2_after = [s2 for s2 in s2_positions if s2 > s1]
        if s2_after:
            interval = s2_after[0] - s1
            # Add interval if it is within the acceptable range
            if min_interval <= interval <= max_interval:
                intervals.append(interval)
    return intervals

# Sampling frequency (Hz) and interval range (ms to samples)
fs = 50  # Sampling frequency
min_interval = int(200 * fs / 1000)  # Convert 200 ms to samples
max_interval = int(500 * fs / 1000)  # Convert 500 ms to samples

# Apply the function to calculate valid S1-S2 intervals
df_probabilities['S1S2_intervals'] = df_probabilities.apply(
    calculate_s1s2_intervals,
    axis=1,
    min_interval=min_interval,
    max_interval=max_interval
)

# Display the updated DataFrame
print(df_probabilities[['S1_positions', 'S2_positions', 'S1S2_intervals']].head())

                                        S1_positions  \
0  [110, 141, 359, 761, 788, 851, 896, 940, 1060,...   
1  [9, 46, 91, 121, 228, 278, 322, 463, 511, 575,...   
2  [20, 66, 110, 156, 201, 247, 308, 337, 380, 42...   
3  [23, 100, 128, 203, 245, 404, 493, 569, 649, 6...   
4  [10, 40, 68, 98, 128, 157, 186, 218, 247, 373,...   

                                        S2_positions  \
0                                             [1431]   
1  [24, 61, 134, 294, 893, 1077, 1124, 1306, 1352...   
2  [37, 82, 173, 218, 263, 352, 397, 442, 532, 57...   
3       [663, 818, 864, 894, 1057, 1186, 1216, 1287]   
4  [22, 51, 80, 110, 139, 168, 198, 230, 260, 385...   

                                      S1S2_intervals  
0                                               [12]  
1           [15, 15, 13, 16, 17, 17, 18, 15, 16, 14]  
2  [17, 16, 17, 17, 16, 15, 17, 17, 17, 17, 17, 1...  
3                       [14, 14, 15, 13, 17, 13, 11]  
4  [12, 11, 12, 12, 11, 11, 12, 12, 13, 12, 12, 1..

In [ ]:
import numpy as np

# Define a function to convert intervals to time and calculate the average
def calculate_avg_interval(row, fs):
    """
    Convert S1S2 intervals to time (ms) and calculate the average.

    Parameters:
    ----------
    row : pandas.Series
        A row of the DataFrame containing S1S2 intervals.
    fs : int
        Sampling frequency in Hz.

    Returns:
    -------
    float
        Average S1S2 interval in milliseconds, or NaN if no intervals exist.
    """
    intervals = row['S1S2_intervals']
    if intervals:  # Ensure there are valid intervals
        intervals_ms = [interval * 1000 / fs for interval in intervals]
        return np.mean(intervals_ms)  # Calculate and return the average in ms
    else:
        return np.nan  # Return NaN for signals with no intervals

# Apply the function to calculate the average S1S2 interval in ms
df_probabilities['Avg_S1S2_interval_ms'] = df_probabilities.apply(
    calculate_avg_interval, axis=1, fs=fs
)

# Display the updated DataFrame
print(df_probabilities[['S1S2_intervals', 'Avg_S1S2_interval_ms']].head())

                                      S1S2_intervals  Avg_S1S2_interval_ms
0                                               [12]            240.000000
1           [15, 15, 13, 16, 17, 17, 18, 15, 16, 14]            312.000000
2  [17, 16, 17, 17, 16, 15, 17, 17, 17, 17, 17, 1...            334.666667
3                       [14, 14, 15, 13, 17, 13, 11]            277.142857
4  [12, 11, 12, 12, 11, 11, 12, 12, 13, 12, 12, 1...            235.200000
